In [5]:
import pandas as pd
#import numpy as np
import pyomo.environ as pe
#from pyomo.opt import SolverFactory
import pyomo.opt as po
import json
import numpy as np
import pprint

In [6]:
inf = 10e7

In [7]:
"""
Datos reales.
"""
#create the model
model = pe.ConcreteModel()
#sets
model.stocks = [i for i in range(1,30)] 
model.sector = ['Technology', 'RawMaterials', 'Health', 'Banking', 'RenewableEnergies']
model.time = [1,2,3,4,5,6]
# Para las acciones que faltan rellenamos con ceros.
# Procesar los datos
price = {}
# Metemos precio
sector_dict = json.load(open("preparado_optimizador.json", "r"))
for i in sector_dict:
    cont = 0
    for j in sector_dict[i]:
        cont += 1
        for k in range(1,7):
            price[(i,cont,k)] = sector_dict[i][j][0][k-1]
    for j in range(len(sector_dict[i])+1,30): # Relleno con ceros, ya que hay sectores que tienen menos de 29 acciones.
        for k in range(1,7):
            price[(i,j,k)] = 0

# Metemos riesgo.
risk = {}
for i in sector_dict:
    cont = 0
    for j in sector_dict[i]:
        cont += 1
        risk[(i,cont)]= sector_dict[i][j][1]
    for j in range(len(sector_dict[i])+1,30): # Relleno con ceros.
        risk[(i,j)] = 0



# open std.json
std_dict = json.load(open("std.json", "r"))


c0 = 1000
budget = {(i): c0+500*i for i in model.time}
# Take the mean of risk 
maxRiskPerStock = np.mean(list(std_dict.values())) + 2*np.std(list(std_dict.values()))   
maxRiskTotal = sum(list(std_dict.values()))/3
maxInvestmentPerStock = {i: budget[i]/3 for i in model.time}

In [8]:
# Aqui empieza el problema de verdad.
model.price = pe.Param(model.sector, model.stocks, model.time, initialize = price)
model.risk = pe.Param(model.sector, model.stocks, initialize = risk)
model.budget = pe.Param(model.time, initialize = budget)
model.mrs = pe.Param(initialize = maxRiskPerStock)
model.mrt = pe.Param(initialize = maxRiskTotal)
model.mips = pe.Param(model.time, initialize = maxInvestmentPerStock)

In [9]:
# Variables
model.buy = pe.Var(model.sector, model.stocks, model.time, within = pe.NonNegativeIntegers)
model.sell = pe.Var(model.sector, model.stocks, model.time, within = pe.NonNegativeIntegers)
model.quantityOwned = pe.Var(model.sector, model.stocks, model.time, within = pe.NonNegativeIntegers)
model.buyBinary = pe.Var(model.sector, model.stocks, model.time, within = pe.Binary)
model.sectorBinary = pe.Var(model.sector, within = pe.Binary)


In [10]:
def obj_rule(model):
    return (sum(model.sell[s, a, t] * model.price[s, a, t] for s in model.sector for a in model.stocks for t in model.time) 
            - sum(model.buy[s, a, t] * model.price[s, a, t] for s in model.sector for a in model.stocks for t in model.time))

model.cost = pe.Objective(rule = obj_rule, sense = pe.maximize)

In [11]:
# Constraints

# Constraint 1: Budget Constraint
model.budgetConstraint = pe.ConstraintList()

for t1 in model.time:
    model.budgetConstraint.add(
        model.budget[t1] + sum(-1 * model.buy[s, a, t2] * model.price[s, a, t2] 
                              + model.sell[s, a, t2] * model.price[s, a, t2] for s in model.sector
                                                                             for a in model.stocks
                                                                             for t2 in model.time if t2 <= t1) >= 0
    )

# Constraint 2: Can't sell what you don't have
model.sellingConstraint = pe.ConstraintList()

for s in model.sector:
    for a in model.stocks:
        for t in model.time:
            if t > 1:
                model.sellingConstraint.add(
                    model.sell[s, a, t] <= model.quantityOwned[s, a, t-1]
                )
            else:
                model.sellingConstraint.add(
                    model.sell[s, a, t] <= 0
                )

# Constraints 3 & 4: Owned Stocks
model.ownedConstraint = pe.ConstraintList()
model.ownedConstraintPositive = pe.ConstraintList()
model.sellEverythingAtEnd = pe.ConstraintList()

for s in model.sector:
    for a in model.stocks:
        for t1 in model.time:
            model.ownedConstraint.add(
                model.quantityOwned[s, a, t1] == sum(model.buy[s, a, t2] - model.sell[s, a, t2] for t2 in model.time if t2 <= t1)
            )
            model.ownedConstraintPositive.add(
                model.quantityOwned[s, a, t1] >= 0
            )

# Constraints 4 & 5: Risk
model.maxRiskPerStock = pe.ConstraintList()

for s in model.sector:
    for a in model.stocks:
        for t in model.time:
            model.maxRiskPerStock.add(
                model.risk[s, a] * model.buyBinary[s, a, t] <= model.mrs
            )

model.maxRiskTotal = pe.ConstraintList()
model.maxRiskTotal.add(sum(model.buyBinary[s, a, t] * model.risk[s, a] for s in model.sector 
                                                                         for a in model.stocks 
                                                                         for t in model.time) <= model.mrt)

# Constraints 6 & 7: Diversification
model.maxInvestmentPerStock = pe.ConstraintList()
model.minDifferentStockAumount = pe.ConstraintList()

for s in model.sector:
    for a in model.stocks:
        for t in model.time:
            model.maxInvestmentPerStock.add(
                model.quantityOwned[s, a, t] * model.price[s, a, t] <= model.mips[t]
            )

# Constraints 8 & 9: Binary Variable Constraints
model.binaryConstraint = pe.ConstraintList()

for s in model.sector:
    for a in model.stocks:
        for t in model.time:
            model.binaryConstraint.add(
                model.buy[s, a, t] <= inf * model.buyBinary[s, a, t]
            )
            model.binaryConstraint.add(
                model.buyBinary[s, a, t] <= inf * model.buy[s, a, t]
            )

# Constraints 10 & 11: Binary Variable Constraints
model.sectorBinaryConstraint = pe.ConstraintList()

for s in model.sector:
    model.sectorBinaryConstraint.add(
        model.sectorBinary[s] <= inf * sum(model.buyBinary[s, a, t] for a in model.stocks for t in model.time)
    )
    model.sectorBinaryConstraint.add(
        sum(model.buyBinary[s, a, t] for a in model.stocks for t in model.time) <= inf * model.sectorBinary[s]
    )


# If you buy from sector Raw Materials, you can't buy from sector Renewable Energy
model.rawRenewableConstraint = pe.ConstraintList()

model.rawRenewableConstraint.add(
    model.sectorBinary['RawMaterials'] + model.sectorBinary['RenewableEnergies'] <= 1
)

# Must invest at least 1 stock on health
model.healthConstraint = pe.ConstraintList()

model.healthConstraint.add(
    model.sectorBinary['Health'] >= 1
)

# We aren't so sure about the banking industry, so we put a harder limit on the maximum risk allowed (Half of the usual allowed)
model.bankingConstraint = pe.ConstraintList()

for s in model.sector:
    if s == 'Banking':
        for a in model.stocks:
            for t in model.time:
                model.bankingConstraint.add(
                    model.risk[s, a] * model.buyBinary[s, a, t] <= 0.5 * model.mrs
                )


In [12]:
solver = po.SolverFactory('glpk')
results = solver.solve(model, tee=True)

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write C:\Users\marta\AppData\Local\Temp\tmp88ln7yku.glpk.raw --wglp C:\Users\marta\AppData\Local\Temp\tmpg30rsqas.glpk.glp
 --cpxlp C:\Users\marta\AppData\Local\Temp\tmpy_s9r2mj.pyomo.lp
Reading problem data from 'C:\Users\marta\AppData\Local\Temp\tmpy_s9r2mj.pyomo.lp'...
C:\Users\marta\AppData\Local\Temp\tmpy_s9r2mj.pyomo.lp:47108: warning: lower bound of variable 'x5229' redefined
C:\Users\marta\AppData\Local\Temp\tmpy_s9r2mj.pyomo.lp:47108: warning: upper bound of variable 'x5229' redefined
6283 rows, 3486 columns, 20346 non-zeros
3485 integer variables, 875 of which are binary
47983 lines were read
Writing problem data to 'C:\Users\marta\AppData\Local\Temp\tmpg30rsqas.glpk.glp'...
39258 lines were written
GLPK Integer Optimizer 5.0
6283 rows, 3486 columns, 20346 non-zeros
3485 integer variables, 875 of which are binary
Preprocessing...
5 hidden covering inequaliti(es) were detected
460 constraint coeffici

In [13]:
print(pe.value(model.cost))

1486.210318837344


In [14]:
df=pd.DataFrame(index = pd.MultiIndex.from_tuples(model.buy,names=['s','a', 't'], ))
df['Stocks Bought'] = [pe.value(model.buy[s, a, t]) for s in model.sector for a in model.stocks for t in model.time]
df['Stocks Sold'] = [pe.value(model.sell[s, a, t]) for s in model.sector for a in model.stocks for t in model.time]

# Save the dataframe to csv
# df.to_csv('resultados_no_constraints.csv')

In [15]:
model.pprint()

40 Set Declarations
    bankingConstraint_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :  174 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174}
    binaryConstraint_index : Size=1, Index=None, Ordered=Insert